In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir(os.getcwd())
from sklearn.metrics import accuracy_score

In [2]:
os.getcwd()

'/mnt/github/RiboVsPolyA/examples'

In [3]:
DEPTH=3

## Using Treehouse Pipeline

In [4]:
srp_tpm_df = pd.read_csv("../results/SRP055411_TPM_log2_plus_1.balanced_maxdepth%d_results.tsv"%DEPTH, index_col=0, sep='\t')

In [5]:
srp_tpm_df.shape

(11, 3)

In [6]:
true_labels = np.array([1]*srp_tpm_df.shape[0])
pred_labels = srp_tpm_df.Ribo.values
pred_proba = srp_tpm_df.Proba_1.values

In [7]:
true_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [8]:
pred_labels

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [9]:
pred_proba

array([0.6366979 , 0.62163716, 0.60653134, 0.63666068, 0.60663716,
       0.5965988 , 0.63166068, 0.60666068, 0.61166068, 0.63895124,
       0.63919404])

In [10]:
acc= accuracy_score(true_labels,pred_labels)
print('Accuracy: {}'.format(acc))

Accuracy: 1.0


## Using refine.bio Pipeline

In [11]:
srp_tpm_df = pd.read_csv("../results/SRP055411_TPM_log2_plus_1.balanced_maxdepth%d_results.tsv"%DEPTH, index_col=0, sep='\t')

In [12]:
srp_tpm_df.shape

(11, 3)

In [13]:
true_labels = np.array([1]*srp_tpm_df.shape[0])
pred_labels = srp_tpm_df.Ribo.values
pred_proba = srp_tpm_df.Proba_1.values

In [14]:
acc= accuracy_score(true_labels,pred_labels)
print('Accuracy: {}'.format(acc))

Accuracy: 1.0
